<b><h1>Disco Data Processing Script</h1>
<h2>Part 1 Reading and Cleaning Data</h2></b>

Prior to running the script, please ensure you have inserted all raw books to be analyzed inside the input directory.
    
<b>For example:</b>    
disco-data-processing.py

input/all raw books in here
     

In [216]:
# importing required libraries:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats


In [3]:
import os 
import glob

#assign local path to raw Excel books
raw_book_path = os.path.abspath('input')
print('Searching in directory:', raw_book_path, '\n')

#list all raw books in file
list_of_raw_books = glob.glob("input/*.xlsx")
print('Raw books to be analyzed are: ', list_of_raw_books, '\n')

Searching in directory: /Users/s.stuart/Documents/Python/disco-data-processing/input 

Raw books to be analyzed are:  ['input/sample_raw.xlsx'] 



In [4]:
#Each loop iteration reviews the data of one raw book (i.e. one polymer) from the input folder
for b in list_of_raw_books:
    
    print("The current book being analyzed is: ", b)
    
    #initialize a "tidy" longform dataframe with important features for analysis i.e. stack one column per variable and one row per observation
    ''' Some Sample Row Data: ['sample', 1,'from 20200130-500KHABSM-4p78125mgml (Second Difference)', 
          9.5625, 0.25, True, 4.21, 2.11, '4.3199..4.1667', 1021.3474],
        ['sample', 1,'from 20200130-500KHABSM-4p78125mgml (Second Difference)', 
          9.5625, 0.25, True, 4.13, 1.9123, '4.1667 .. 4.0025', 914.0484],'''
    
    current_book_clean = pd.DataFrame([], columns = ['sample_or_control',
                   'replicate_index',
                    'title_string',
                    'concentration',
                    'sat_time',
                    'irradiation_bool',
                    'peak_ppm',
                    'peak_intensity',
                    'peak_range',
                    'peak_absolute_integral']) 
    
    #determine the number of sheets, samples, & controls in the workbook
    name_sheets = pd.ExcelFile(b).sheet_names
    num_sheets = (len(pd.ExcelFile(b).sheet_names))
    
    #initialize number of samples and controls to zero, then initialize the "list initializers" which will become global lists that feed into the clean dataframe to create the clean frame's global structure.
    num_samples = 0
    num_controls = 0
    sample_control_initializer = []
    sample_replicate_initializer = []
    control_replicate_initializer = []

    #loop through sheet names to determine number of samples and controls
    for s in range(len(name_sheets)):
        #if the current sheet is labeled Sample: 
        #increment sample counter, add a list item called 'sample' to be initialized into the 'sample_or_control' column, add a list item of the replicate number to be initialized into the 'replicate' column in the clean dataframe later.
        if ('Sample' or 'sample') in name_sheets[s]:
            num_samples += 1
            sample_control_initializer.append('sample')
            sample_replicate_initializer.append(num_samples)
            
        #if the current sheet is labeled Control: 
        #increment control counter, add a list item called 'control' to be initialized into the 'sample_or_control' column, add a list item of the replicate number to be initialized into the 'replicate' column in the clean dataframe later.   
        elif ('Control' or 'control') in name_sheets[s]:
            num_controls += 1
            sample_control_initializer.append('control')
            control_replicate_initializer.append(num_controls)
    
    print("There are", num_sheets, "sheets identified in the current book.")
    print("Number of samples sheets identified:", num_samples)
    print("Number of control sheets identified:", num_controls)
    
    if num_samples != num_controls:
        print('\nERROR: The number of sample sheets is not equal to the number of control sheets in', b, 'please confirm the data in the book is correct.')
    
    #combine sample and control initializers to create the total replicate list to be fed into the clean data frame under 'replicate_index'
    total_replicate_index = sample_replicate_initializer + control_replicate_initializer
    
    #assign the list created of the identified samples and controls and their replicates to the clean dataframe to initialize the proper 'sample_or_control' and 'replicate_index' columns
    current_book_clean['sample_or_control'] = sample_control_initializer
    current_book_clean['replicate_index'] = total_replicate_index
    
    print("The 'sample_or_control' column in the Clean dataframe has been initialized to:", sample_control_initializer)
    print("The 'replicate_index' column in the Clean dataframe has been initialized to:", total_replicate_index)
    print("Book-level clean data initializations complete. Beginning looping through replicate-specific data.")

The current book being analyzed is:  input/sample_raw.xlsx
There are 6 sheets identified in the current book.
Number of samples sheets identified: 3
Number of control sheets identified: 3
The 'sample_or_control' column in the Clean dataframe has been initialized to: ['sample', 'sample', 'sample', 'control', 'control', 'control']
The 'replicate_index' column in the Clean dataframe has been initialized to: [1, 2, 3, 1, 2, 3]
Book-level clean data initializations complete. Beginning looping through replicate-specific data.


In [5]:
#NOTE - THIS SECTION AND ALL THAT COMES AFTER IT SHOULD STILL GO INSIDE THE BOOK LEVEL FOR LOOP WHEN ALL COMPLETE SO THAT THE CODE IS GENERALIZABLE TO MULTIPLE BOOKS 

'''The next column label to initialize in the clean dataframe are the 'title_strings' for each replicate.'''

#initialize 'title_string' column initializer, and the rows initializer for the new dataframe.
title_string_initializer = []
title_string_rows = []
col_index = []

#loop through each sheet in the workbook
for n in range(num_sheets):
    
    #read the current book's nth sheet into a Pandas dataframe
    current_sheet_raw = pd.read_excel(b, sheet_name=n)
    
    #assign what sample or control and replicate the book's nth sheet is
    current_sample_or_control = sample_control_initializer[n]
    current_replicate_index = total_replicate_index[n]
    
    #search the first row of each sheet and append the 'title_string_initializer' with a title string there is a match
    for col in current_sheet_raw:
        if "from" in col:
            title_string_initializer.append(col)
            
            #assign col index to list
            
            current_title_string_initializer = title_string_initializer[len(title_string_initializer)-1]
            #fill in the current data for the nth sheet
            title_string_rows.append([current_sample_or_control, current_replicate_index, current_title_string_initializer])
        
clean_df = pd.DataFrame(title_string_rows, columns=['sample_or_control','replicate_index','title_string'])
            
#determine whether or not to replace the clean book with this dataframe later.... it worked!
clean_df


sample_or_control  replicate_index  \
0             sample                1   
1             sample                1   
2             sample                1   
3             sample                1   
4             sample                1   
5             sample                2   
6             sample                2   
7             sample                2   
8             sample                2   
9             sample                2   
10            sample                3   
11            sample                3   
12            sample                3   
13            sample                3   
14            sample                3   
15           control                1   
16           control                1   
17           control                1   
18           control                1   
19           control                1   
20           control                2   
21           control                2   
22           control                2   
23           control                2   
24           control                2   
25           control                3   
26           control                3   
27           control                3   
28           control                3   
29           control                3   

                                         title_string  
0   from 20200130-500KHABSM-4p78125mgml (Second Di...  
1   from 20200130-500KHABSM-6p750mgml (Second Diff...  
2   from 20200130-500KHABSM-5p625mgml (Second Diff...  
3   from 20200130-500KHABSM-4p064mgml (Second Diff...  
4   from 20200130-500KHABSM-3p450mgml (Second Diff...  
5   from 20200130-2-500KHABSM-4p78125mgml (Second ...  
6   from 20200130-2-500KHABSM-6p750mgml (Second Di...  
7   from 20200130-2-500KHABSM-5p625mgml (Second Di...  
8   from 20200130-2-500KHABSM-4p064mgml (Second Di...  
9   from 20200130-2-500KHABSM-3p450mgml (Second Di...  
10  from 20200130-3-500KHABSM-4p78125mgml (Second ...  
11  from 20200130-3-500KHABSM-6p750mgml (Second Di...  
12  from 20200130-3-500KHABSM-5p625mgml (Second Di...  
13  from 20200130-3-500KHABSM-4p064mgml (Second Di...  
14  from 20200130-3-500KHABSM-3p450mgml (Second Di...  
15                   from 20200130-500KHA-4p78125mgml  
16                     from 20200130-500KHA-6p750mgml  
17                     from 20200130-500KHA-5p625mgml  
18                     from 20200130-500KHA-4p064mgml  
19                     from 20200130-500KHA-3p450mgml  
20                 from 20200130-2-500KHA-4p78125mgml  
21                   from 20200130-2-500KHA-6p750mgml  
22                   from 20200130-2-500KHA-5p625mgml  
23                   from 20200130-2-500KHA-4p064mgml  
24                   from 20200130-2-500KHA-3p450mgml  
25                 from 20200130-3-500KHA-4p78125mgml  
26                   from 20200130-3-500KHA-6p750mgml  
27                   from 20200130-3-500KHA-5p625mgml  
28                   from 20200130-3-500KHA-4p064mgml  
29                   from 20200130-3-500KHA-3p450mgml

In [187]:
#FOR TEST PURPOSES SET NUM SHEETS TO 1
num_sheets=1

#loop through each sheet in the workbook
for n in range(num_sheets):
    #read in the current book's nth sheet into a Pandas dataframe
    current_sheet_raw = pd.read_excel(b, sheet_name=n)
    
    #drop first always empty col
    to_drop = ['Unnamed: 0']
    current_sheet_raw.drop(columns = to_drop, inplace = True)

    #loop through columns and "fill right" to replace all Unnamed columns with the right title_string value
    for c in current_sheet_raw.columns:
        current_sheet_raw.columns = [current_sheet_raw.columns[i-1] if 'Unnamed' in current_sheet_raw.columns[i] else current_sheet_raw.columns[i] for i in range(len(current_sheet_raw.columns))]

      

In [312]:
#initialize indices
conc_indices = []
conc_columns = []

#initialize dfs
current_exp_df = []
df_list = []
big_df = []


#identifies the coordinates of the left-most parameter in each experimental set, conc (um)
i, c = np.where(current_sheet_raw == 'conc (um)')

#assigns coordinates of all 'conc (um) cells to an index (row) array and a column array'
conc_indices = current_sheet_raw.index.values[i]
conc_columns = current_sheet_raw.columns.values[c]

#determine the number of experimental rows in each NMR results set, applies to whole book
#subtract one to exclude conc itself
num_experimental_indices = np.unique(conc_indices)[2] - np.unique(conc_indices)[1] - 1
# print("The number of experimental indices are:", num_experimental_indices)

#determine the number of experimental columns in each NMR results set, applies to whole book
(unique_exp_cols, count_experimental_cols) = np.unique(conc_columns, return_counts = True)
num_experimental_cols = np.unique(count_experimental_cols)[0]
# print("The number of experimental columns are:", num_experimental_cols)

#within the current_sheet_raw, [i,j] will be the row and column indices of the concentration cell for each experimental set
for i in conc_indices:
    for j in conc_columns:
        
        #dummy values, fill in logic from above code later
        current_sample_or_control = 'sample'
        current_replicate = 1
        
        #assigns current values to the fixed experimental parameters for this set
        fixed_parameters_per_set = current_sheet_raw.loc[i,j]
        
        current_title_string = fixed_parameters_per_set.index[0]
        current_concentration = fixed_parameters_per_set[1]
        current_sat_time = fixed_parameters_per_set[3]
        current_irrad_bool = fixed_parameters_per_set[5]
        
        #initialize/reset experimental lists to null for this experimental set
        list_current_ppm_experimental = []
        list_current_intensity_experimental = []
        list_current_width_experimental = []
        list_current_area_experimental = []
        list_current_type_experimental = []
        list_current_flags_experimental = []
        list_current_impuritycomp_experimental = []
        list_current_annotation_experimental = []
        list_current_range_experimental = []
        list_current_normalized_experimental = []
        list_current_absolute_experimental = []
        
        #  now need to find and assign experimental values to new lists, in a generalizable way
        experimental_index_starter = i+1
#         print("Experimental Index Starter is:", experimental_index_starter)
        
        experimental_index_range = range(num_experimental_indices)
#         print("Experimental Index Range is:", experimental_index_range)
        
        experimental_column_range = range(num_experimental_cols)
#         print("Experimental Column Range is:", experimental_index_range)
        
        combined_experimental_index_range = experimental_index_starter + experimental_index_range
#         print("Combined Experimental Index Range is:", combined_experimental_index_range)
        
        #Obtain experimental column range using boolean mask as column labels are title_strings for each experimental set
        experimental_column_range_mask = current_sheet_raw.columns.get_loc(j)
        combined_experimental_column_range = np.where(experimental_column_range_mask)
#         print("Combined Experimental Column Range is:", combined_experimental_column_range)
        
        #loop through and collect experimental index and experimental column range for the ith and jth experimental set
        for ei in combined_experimental_index_range:
            for ec in combined_experimental_column_range:
                
#               use iloc to grab NMR experimental variables for this set into a series
                variable_parameters_per_set = current_sheet_raw.iloc[ei,ec]
#                 print(variable_parameters_per_set)
            
#               append designated variable parameters from the generated Series' into lists for those parameters for this experimental set 
                list_current_ppm_experimental.append(variable_parameters_per_set.iloc[1])
                list_current_intensity_experimental.append(variable_parameters_per_set.iloc[2])
                list_current_width_experimental.append(variable_parameters_per_set.iloc[3])
                list_current_area_experimental.append(variable_parameters_per_set.iloc[4])
                list_current_type_experimental.append(variable_parameters_per_set.iloc[5])
                list_current_flags_experimental.append(variable_parameters_per_set.iloc[6])
                list_current_impuritycomp_experimental.append(variable_parameters_per_set.iloc[7])
                list_current_annotation_experimental.append(variable_parameters_per_set.iloc[8])
                list_current_range_experimental.append(variable_parameters_per_set.iloc[11])
                list_current_normalized_experimental.append(variable_parameters_per_set.iloc[12])
                list_current_absolute_experimental.append(variable_parameters_per_set.iloc[13])

#                 FOR TESTING
#                 list_current_ppm_experimental = [1, 2, 3, 4, 5]
#                 list_current_intensity_experimental = [1, 2, 3, 4, 5]
#                 list_current_width_experimental = ['test', 'test', 'test', 'test', 'test']
#                 list_current_area_experimental = ['test', 'test', 'test', 'test', 'test']
#                 list_current_type_experimental = ['test', 'test', 'test', 'test', 'test']
#                 list_current_flags_experimental = ['test', 'test', 'test', 'test', 'test']
#                 list_current_impuritycomp_experimental = ['test', 'test', 'test', 'test', 'test']
#                 list_current_annotation_experimental = ['test', 'test', 'test', 'test', 'test']
#                 list_current_range_experimental = ['test', 'test', 'test', 'test', 'test']
#                 list_current_normalized_experimental = ['test', 'test', 'test', 'test', 'test']
#                 list_current_absolute_experimental = [1, 2, 3, 4, 5]
                
        #after all the experimental lists are populated, define length of the experimental parameter lists (number of true experimental rows)
        exp_list_length = len(list_current_ppm_experimental)
#                 print('Experimental List Length is:', exp_list_length)
                
        #create "range" lists of the fixed experimental values to be the same length as variable experimental values, so we can add both to the same dataframe
        ranged_sample_or_control = exp_list_length * [current_sample_or_control]
        ranged_replicate = exp_list_length * [current_replicate]
        ranged_title_string = exp_list_length * [current_title_string]
        ranged_concentration = exp_list_length * [current_concentration] 
        ranged_sat_time = exp_list_length * [current_sat_time]
        ranged_irrad_bool = exp_list_length * [current_irrad_bool]
        
    
        #assign all current experimental values for this experimental set to a dataframe
        current_exp_df = pd.DataFrame(list(zip(ranged_sample_or_control, ranged_replicate, ranged_title_string, 
                                                      ranged_concentration, ranged_sat_time, ranged_irrad_bool,
                                                      list_current_ppm_experimental, list_current_intensity_experimental,
                                                      list_current_width_experimental, list_current_area_experimental,
                                                      list_current_type_experimental, list_current_flags_experimental, 
                                                      list_current_impuritycomp_experimental, list_current_annotation_experimental,
                                                      list_current_range_experimental, list_current_normalized_experimental, 
                                                      list_current_absolute_experimental)),
                                              
                                             columns = ['sample_or_control', 
                                                        'replicate', 
                                                        'title_string', 
                                                        'concentration', 
                                                        'sat_time', 
                                                        'irrad_bool', 
                                                        'ppm', 'intensity',
                                                        'width', 'area', 'type',
                                                        'flags', 'impurity_compound',
                                                        'annotation', 'range', 'normalized', 'absolute'])
            
            
        #before moving on to next experimental set, append the dataframe from this experimental set to a global list of dataframes
        df_list.append(current_exp_df)
    
    print("Data frame for the experimental set at coordinates:", i, j, "has been appended to the global list of dataframes.")
        
#concatenate the global list of dataframes into one big beautiful clean dataframe!
big_df = pd.concat(df_list)
big_df.to_excel("bigdf.xlsx")
print("Done - global list has been concatenated into one big beautiful dataframe, and exported to Excel!")

#reintegrate original code about replicate and sample into this
big_df        

Data frame for the experimental set at coordinates: 1 from 20200130-500KHABSM-3p450mgml (Second Difference) has been appended to the global list of dataframes.
Data frame for the experimental set at coordinates: 1 from 20200130-500KHABSM-3p450mgml (Second Difference) has been appended to the global list of dataframes.
Data frame for the experimental set at coordinates: 1 from 20200130-500KHABSM-3p450mgml (Second Difference) has been appended to the global list of dataframes.
Data frame for the experimental set at coordinates: 1 from 20200130-500KHABSM-3p450mgml (Second Difference) has been appended to the global list of dataframes.
Data frame for the experimental set at coordinates: 1 from 20200130-500KHABSM-3p450mgml (Second Difference) has been appended to the global list of dataframes.
Data frame for the experimental set at coordinates: 10 from 20200130-500KHABSM-3p450mgml (Second Difference) has been appended to the global list of dataframes.
Data frame for the experimental set at 

Data frame for the experimental set at coordinates: 91 from 20200130-500KHABSM-3p450mgml (Second Difference) has been appended to the global list of dataframes.
Data frame for the experimental set at coordinates: 91 from 20200130-500KHABSM-3p450mgml (Second Difference) has been appended to the global list of dataframes.
Data frame for the experimental set at coordinates: 91 from 20200130-500KHABSM-3p450mgml (Second Difference) has been appended to the global list of dataframes.
Data frame for the experimental set at coordinates: 91 from 20200130-500KHABSM-3p450mgml (Second Difference) has been appended to the global list of dataframes.
Data frame for the experimental set at coordinates: 100 from 20200130-500KHABSM-3p450mgml (Second Difference) has been appended to the global list of dataframes.
Data frame for the experimental set at coordinates: 100 from 20200130-500KHABSM-3p450mgml (Second Difference) has been appended to the global list of dataframes.
Data frame for the experimental 

sample_or_control  replicate  \
0             sample          1   
1             sample          1   
2             sample          1   
3             sample          1   
4             sample          1   
..               ...        ...   
3             sample          1   
4             sample          1   
5             sample          1   
6             sample          1   
7             sample          1   

                                         title_string  concentration  \
0   from 20200130-500KHABSM-4p78125mgml (Second Di...         9.5625   
1   from 20200130-500KHABSM-4p78125mgml (Second Di...         9.5625   
2   from 20200130-500KHABSM-4p78125mgml (Second Di...         9.5625   
3   from 20200130-500KHABSM-4p78125mgml (Second Di...         9.5625   
4   from 20200130-500KHABSM-4p78125mgml (Second Di...         9.5625   
..                                                ...            ...   
3   from 20200130-500KHABSM-3p450mgml (Second Diff...         6.9000   
4   from 20200130-500KHABSM-3p450mgml (Second Diff...         6.9000   
5   from 20200130-500KHABSM-3p450mgml (Second Diff...         6.9000   
6   from 20200130-500KHABSM-3p450mgml (Second Diff...         6.9000   
7   from 20200130-500KHABSM-3p450mgml (Second Diff...         6.9000   

    sat_time  irrad_bool   ppm  intensity    width         area      type  \
0       0.25        True   ppm  Intensity    Width         Area      Type   
1       0.25        True  4.21       2.11    26.94      1020.35  Compound   
2       0.25        True  4.13     1.9123    23.86       819.22  Compound   
3       0.25        True  3.51     9.5708  9525.91  1.63681e+06  Compound   
4       0.25        True  3.19    11.4251    27.85       5711.8  Compound   
..       ...         ...   ...        ...      ...          ...       ...   
3       1.75       False  3.51     8.3377    27.06       4050.6  Compound   
4       1.75       False  3.19    10.1104    26.83      4869.91  Compound   
5       1.75       False  3.01     4.4744    32.99      2650.45  Compound   
6       1.75       False  1.69    52.0757     4.24      3967.42  Compound   
7       1.75       False  0.84     9.9089     1.95       346.87  Compound   

    flags  impurity_compound  annotation             range  normalized  \
0   Flags  Impurity/Compound  Annotation             Range  Normalized   
1    None                NaN         NaN  4.3199 .. 4.1667        2.96   
2    None                NaN         NaN  4.1667 .. 4.0025        2.65   
3    None                NaN         NaN  3.5985 .. 3.4369       11.55   
4    None                NaN         NaN  3.2205 .. 3.1641        9.71   
..    ...                ...         ...               ...         ...   
3    None                NaN         NaN  3.5985 .. 3.4369       12.81   
4    None                NaN         NaN  3.2205 .. 3.1641       10.87   
5    None                NaN         NaN  3.0701 .. 2.9498        7.87   
6    None                NaN         NaN  1.7148 .. 1.6698       14.27   
7    None                NaN         NaN  0.8491 .. 0.8417        1.02   

    absolute  
0   Absolute  
1    1021.35  
2    914.048  
3    3984.77  
4    3350.94  
..       ...  
3    3450.81  
4    2928.09  
5    2120.02  
6    3845.63  
7    274.456  

[39200 rows x 17 columns]

In [ ]:
mask = (big_df.index == 0)
print(mask)
# big_df.drop(mask)
# print(big_df)

In [ ]:
#define function to call during apply map to generate a clean dataframe for each experimental set
#submit to stack overflow tomorrow??


def get_column_name1(x):
    return x.name

def convert_nmr_sets_to_dataframes(item):
    #identifies the left-most parameter in each experimental set, conc (um)
    if 'conc (um)' == item:
        
        #dummy values, fill in logic from above code later
        current_sample_or_control = 'sample'
        current_replicate = 1
        i, c = np.where(current_sheet_raw == item)
        
        #assigns current values to the fixed experimental parameters for this set
#         current_title_string = current_sheet_raw.columns
#         current_concentration = 0 
#         current_time = 0
#         current_irrad_bool = 0
        
        #loop below item
        
        return item
    
current_sheet_raw_test =  current_sheet_raw.applymap(convert_nmr_sets_to_dataframes)
# current_sheet_raw_test.to_excel("testingfunction.xlsx")

i, c

In [31]:
#define function that returns transposed dataframe row contents that match a search query
def grab_content(df,query):  
    #transpose dataframe
    df_transpose = df.transpose()
    #identify where query is true for the transpose dataframe
    query_match_transpose = (df_transpose == query).any()
    #get list of df indices where the query match is true 
    iloc_list = query_match_transpose[query_match_transpose].index.values
    #grab contents of the row in the transposed dataframe where the queries match
    grabbed_content = df.iloc[iloc_list]
    return grabbed_content

def grab_content_n_below(df,query,n):  
    #transpose dataframe
    df_transpose = df.transpose()
    #identify where query is true for the transpose dataframe
    query_match_transpose = (df_transpose == query).any()
    #get list of df indices where the query match is true 
    iloc_list = query_match_transpose[query_match_transpose].index.values
    #obtain the indices for the nth below row
    iloc_list_n = iloc_list + n
    #grab contents of the row in the transposed dataframe nth row below where the queries match
    grabbed_content_n_below = df.iloc[iloc_list_n]
    return grabbed_content_n_below

#Initialize empty list for dataframes
list_of_dfs = []

#Define query to indicate rows to start grabbing data from:
grab_query = "conc (um)"

#FOR TEST PURPOSES SET NUM SHEETS TO 1
num_sheets=1

#loop through each sheet in the workbook
for n in range(num_sheets):
    
    #read in the current book's nth sheet into a Pandas dataframe
    current_sheet_raw = pd.read_excel(b, sheet_name=n)
    
    #Read the conc (um), time(s), and irrad(t/f) data from the nth sheet through the grab content function
    current_df = grab_content(current_sheet_raw, grab_query)
    current_df_dropna = current_df.dropna(axis = 1)
    
    #grab the title string boolean series, assign match key to existing df
    title_string_bool = (current_df == grab_query).any()
    loc_list = title_string_bool[title_string_bool].index.values
    
    #loop through columns and "fill right" to replace Unnamed columns with title_string value
    for c in current_df_dropna.columns:
        current_df_dropna.columns = [current_df_dropna.columns[i-1] if 'Unnamed' in current_df_dropna.columns[i] else current_df_dropna.columns[i] for i in range(len(current_df_dropna.columns))]

    #iterate through rows in the completely labeled df, then assign known parameters into a fresh, organized dataframe for that sheet   
    for index, row in current_df_dropna.iterrows():
        row = next(current_df_dropna.iterrows())[1]

#PLAN TO PROCEED:
    
#         new_df = pd.DataFrame([row.index], columns = ['title_string'])
        
#title-string, will correspond to each of those indices
row.index

# #concentration
# print(row[0])
# print(row[1])
# #time
# print(row[2])
# print(row[3])
# #irrad
# print(row[4])
# print(row[5])

#add +5 to those indices and it's the whole enchilada for this sheet
#once current sheet nicely in a dataframe, add that dataframe to a list of dataframes
#once a list of all dfs from book complete, concatenate those
#then learn how to db merge the previous df_clean and the nicely labeled dataframes from this exercise using the title as the principal key?

# current_df_dropna


#iterate through rows in the raw df, then assign known parameters into a fresh, organized dataframe for that sheet   
for index, row in current_sheet_raw.iterrows():
    row = next(current_sheet_raw.iterrows())
#     print((row))
        
#concentration
# print(row)
# print(row[1])
# #time
# print(row[2])
# print(row[3])
# #irrad
# print(row[4])
# print(row[5])



from 20200130-500KHABSM-4p78125mgml (Second Difference)  \
0                                                NaN         
1                                          conc (um)         
2                                                NaN         
3                                                  1         
4                                                  2         

   from 20200130-500KHABSM-4p78125mgml (Second Difference)  \
0                                                NaN         
1                                             9.5625         
2                                                ppm         
3                                               4.21         
4                                               4.13         

   from 20200130-500KHABSM-4p78125mgml (Second Difference)  \
0                                                NaN         
1                                            time(s)         
2                                          Intensity         
3                                               2.11         
4                                             1.9123         

   from 20200130-500KHABSM-4p78125mgml (Second Difference)  \
0                                                NaN         
1                                               0.25         
2                                              Width         
3                                              26.94         
4                                              23.86         

   from 20200130-500KHABSM-4p78125mgml (Second Difference)  \
0                                                NaN         
1                                        irrad (t/f)         
2                                               Area         
3                                            1020.35         
4                                             819.22         

   from 20200130-500KHABSM-4p78125mgml (Second Difference)  \
0                                                NaN         
1                                               True         
2                                               Type         
3                                           Compound         
4                                           Compound         

   from 20200130-500KHABSM-4p78125mgml (Second Difference)  \
0                                                NaN         
1                                                NaN         
2                                              Flags         
3                                               None         
4                                               None         

   from 20200130-500KHABSM-4p78125mgml (Second Difference)  \
0                                                NaN         
1                                                NaN         
2                                  Impurity/Compound         
3                                                NaN         
4                                                NaN         

   from 20200130-500KHABSM-4p78125mgml (Second Difference)  \
0                                                NaN         
1                                                NaN         
2                                         Annotation         
3                                                NaN         
4                                                NaN         

   from 20200130-500KHABSM-4p78125mgml (Second Difference)  ...  \
0                                                NaN        ...   
1                                                NaN        ...   
2                                                NaN        ...   
3                                                NaN        ...   
4                                                NaN        ...   

  from 20200130-500KHABSM-3p450mgml (Second Difference)  \
0                                                NaN      
1                                        irrad (t/f)      
2                                               Area      
3      

In [305]:
#define function that returns a transposed dataframe row contents that match a search query
def grab_content(df,query):  
    #transpose dataframe
    df_transpose = df.transpose()
    #identify where query is true for the transpose dataframe
    query_match_transpose = (df_transpose == query).any()
    #get list of df indices where the query match is true 
    iloc_list = query_match_transpose[query_match_transpose].index.values
    #grab contents of the row in the transposed dataframe where the queries match
    grabbed_content = df.iloc[iloc_list]
    return grabbed_content

def grab_content_n_below(df,query,n):  
    #transpose dataframe
    df_transpose = df.transpose()
    #identify where query is true for the transpose dataframe
    query_match_transpose = (df_transpose == query).any()
    #get list of df indices where the query match is true 
    iloc_list = query_match_transpose[query_match_transpose].index.values
    #obtain the indices for the nth below row
    iloc_list_n = iloc_list + n
    #grab contents of the row in the transposed dataframe nth row below where the queries match
    grabbed_content_n_below = df.iloc[iloc_list_n]
    return grabbed_content_n_below

#Initialize empty list for dataframes
list_of_dfs = []
list_of_dfs_n_below = []
list_of_dfs_test = []
rows_below = 7

#Define query to indicate rows to start grabbing data from:
grab_query = "conc (um)"

#loop through each sheet in the workbook
for n in range(num_sheets):
    
    #read in the current book's nth sheet into a Pandas dataframe
    current_sheet_raw = pd.read_excel(b, sheet_name=n)
    
    #Appendthe conc (um), time(s), and irrad(t/f) data from the nth sheet through the grab content function
    list_of_dfs.append(grab_content(current_sheet_raw, grab_query))
    list_of_dfs_n_below.append(grab_content_n_below(current_sheet_raw, grab_query, rows_below))
#     print(len(list_of_dfs))

#concatenate list of dataframes into new dataframe containing information from all sheets
conc_time_irrad_data = pd.concat(list_of_dfs)
conc_time_irrad_data.to_excel("conc_time_irrad_data_concatenated.xlsx")

conc_n_below_data = pd.concat(list_of_dfs_n_below)
conc_n_below_data.to_excel("conc_time_irrad_data_concatenated_n_below.xlsx")

In [287]:
# function to return boolean values of spreadsheet columns based on whether they contain an exact query word
def my_finder_function(df,query):  
    return (df == query).any()

def my_finder_to_the_right_function(df,query):  

    #transpose query_match
    df_transpose = df.transpose()
    
    #identify where query is true for the transpose
    query_match_transpose = (df_transpose == query).any()
    
    #get list of df indices where boolean is true (which iloc uses to identify)
    iloc_list = query_match_transpose[query_match_transpose].index.values
    
    #get list of df indices to the right where the desired information is
    
    #this goes one row below instead of one row to the right...
#     iloc_list_to_the_right = iloc_list + 1
    
#     loc_list = [i for i in query_match_trans.index if query_match[i]]
    #grab contents to the right (+1) of the indices where boolean true, and return that, using loc
    contents_to_the_right = df.iloc[iloc_list]

    return contents_to_the_right

#queries for key parameters:

#based on speadsheet layout, num will be one to the right of these cols
concentration_query = "conc (um)"
time_query = "time(s)"
irrad_bool_query = "irrad (t/f)"

#based on spreadsheet layout, num will be in these cols
ppm_query = "ppm"
intensity_query = "Intensity"
range_query = "Range"
absolute_query = "Absolute"

#assign boolean series to variables that store which columns contain the desired queries
concentration_label_cols = my_finder_function(current_sheet_raw, concentration_query)
time_label_cols = my_finder_function(current_sheet_raw, time_query)
irrad_bool_cols = my_finder_function(current_sheet_raw, irrad_bool_query)
ppm_cols = my_finder_function(current_sheet_raw, ppm_query)
intensity_cols = my_finder_function(current_sheet_raw, intensity_query)
range_cols = my_finder_function(current_sheet_raw, range_query)
absolute_cols = my_finder_function(current_sheet_raw, absolute_query)

#Transpose raw sheet to complete subsetting operations based on boolean series
current_sheet_raw_transpose = current_sheet_raw.transpose()

# true_conc = current_sheet_raw_transpose[current_sheet_raw_transpose[concentration_label_cols]]

#concatenate the true booleans into dataframes
series_numtotheright = [concentration_label_cols,time_label_cols,irrad_bool_cols]
series_numincolumn = [ppm_cols,intensity_cols,range_cols, absolute_cols]
df_numtotheright = pd.concat(series_numtotheright)
df_numincolumn = pd.concat(series_numincolumn)



testing = my_finder_to_the_right_function(current_sheet_raw, "conc (um)")
testing_mask = testing.dropna(axis = 1)
testing_mask



from 20200130-3-500KHA-4p78125mgml Unnamed: 2 Unnamed: 3 Unnamed: 4  \
1                            conc (um)     9.5625    time(s)       0.25   
10                           conc (um)     9.5625    time(s)       0.25   
19                           conc (um)     9.5625    time(s)        0.5   
28                           conc (um)     9.5625    time(s)        0.5   
37                           conc (um)     9.5625    time(s)       0.75   
46                           conc (um)     9.5625    time(s)       0.75   
55                           conc (um)     9.5625    time(s)          1   
64                           conc (um)     9.5625    time(s)          1   
73                           conc (um)     9.5625    time(s)       1.25   
82                           conc (um)     9.5625    time(s)       1.25   
91                           conc (um)     9.5625    time(s)        1.5   
100                          conc (um)     9.5625    time(s)        1.5   
109                          conc (um)     9.5625    time(s)       1.75   
118                          conc (um)     9.5625    time(s)       1.75   

      Unnamed: 5 Unnamed: 6 from 20200130-3-500KHA-6p750mgml  Unnamed: 17  \
1    irrad (t/f)       True                        conc (um)         13.5   
10   irrad (t/f)      False                        conc (um)         13.5   
19   irrad (t/f)       True                        conc (um)         13.5   
28   irrad (t/f)      False                        conc (um)         13.5   
37   irrad (t/f)       True                        conc (um)         13.5   
46   irrad (t/f)      False                        conc (um)         13.5   
55   irrad (t/f)       True                        conc (um)         13.5   
64   irrad (t/f)      False                        conc (um)         13.5   
73   irrad (t/f)       True                        conc (um)         13.5   
82   irrad (t/f)      False                        conc (um)         13.5   
91   irrad (t/f)       True                        conc (um)         13.5   
100  irrad (t/f)      False                        conc (um)         13.5   
109  irrad (t/f)       True                        conc (um)         13.5   
118  irrad (t/f)      False                        conc (um)         13.5   

    Unnamed: 18  Unnamed: 19  ... Unnamed: 48  Unnamed: 49  Unnamed: 50  \
1       time(s)         0.25  ...     time(s)         0.25  irrad (t/f)   
10      time(s)         0.25  ...     time(s)         0.25  irrad (t/f)   
19      time(s)         0.50  ...     time(s)          0.5  irrad (t/f)   
28      time(s)         0.50  ...     time(s)          0.5  irrad (t/f)   
37      time(s)         0.75  ...     time(s)         0.75  irrad (t/f)   
46      time(s)         0.75  ...     time(s)         0.75  irrad (t/f)   
55      time(s)         1.00  ...     time(s)            1  irrad (t/f)   
64      time(s)         1.00  ...     time(s)            1  irrad (t/f)   
73      time(s)         1.25  ...     time(s)         1.25  irrad (t/f)   
82      time(s)         1.25  ...     time(s)         1.25  irrad (t/f)   
91      time(s)         1.50  ...     time(s)          1.5  irrad (t/f)   
100     time(s)         1.50  ...     time(s)          1.5  irrad (t/f)   
109     time(s)         1.75  ...     time(s)         1.75  irrad (t/f)   
118     time(s)         1.75  ...     time(s)         1.75  irrad (t/f)   

     Unnamed: 51 from 20200130-3-500KHA-3p450mgml  Unnamed: 62 Unnamed: 63  \
1           True                        conc (um)          6.9     time(s)   
10         False                        conc (um)          6.9     time(s)   
19          True                        conc (um)          6.9     time(s)   
28         False                        conc (um)          6.9     time(s)   
37          True                        conc (um)          6.9     time(s)   
46         False                        conc (um)          6.9     time(s)   
55          True                        conc (um) 

In [314]:
#this is really good
df = current_sheet_raw

for label, content in df.items():
    
#     if df[label[0]].str.contains('from').any():
        print('label:', label)
        print('content:', content, sep='\n')
    
    

label: Unnamed: 0
content:
0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
       ..
122   NaN
123   NaN
124   NaN
125   NaN
126   NaN
Name: Unnamed: 0, Length: 127, dtype: float64
label: from 20200130-3-500KHA-4p78125mgml
content:
0            NaN
1      conc (um)
2            NaN
3              1
4              2
         ...    
122            3
123            4
124            5
125            6
126            7
Name: from 20200130-3-500KHA-4p78125mgml, Length: 127, dtype: object
label: Unnamed: 2
content:
0         NaN
1      9.5625
2         ppm
3        4.21
4        4.13
        ...  
122      3.51
123      3.19
124      3.01
125      1.69
126      0.84
Name: Unnamed: 2, Length: 127, dtype: object
label: Unnamed: 3
content:
0            NaN
1        time(s)
2      Intensity
3         2.2258
4         1.9984
         ...    
122       11.238
123      13.5688
124        5.888
125      71.1131
126      14.7875
Name: Unnamed: 3, Length: 127, dtype: object
label: Unnamed: 4
content

In [140]:
df = pd.DataFrame({'species': ['mammal', 'mammal', 'fish'],
                   'population': [3948, 4000, 6000]},
                  index=['tiger', 'fox', 'shark'])

for label, content in df.items():
    print('label:', label)
    print('content:', content, sep='\n')

label: species
content:
tiger    mammal
fox      mammal
shark      fish
Name: species, dtype: object
label: population
content:
tiger    3948
fox      4000
shark    6000
Name: population, dtype: int64


In [315]:
for index, row in current_sheet_raw.iterrows():
    row = next(current_sheet_raw.iterrows())

print(row)    

(0, Unnamed: 0                            NaN
from 20200130-3-500KHA-4p78125mgml    NaN
Unnamed: 2                            NaN
Unnamed: 3                            NaN
Unnamed: 4                            NaN
                                     ... 
Unnamed: 70                           NaN
Unnamed: 71                           NaN
Unnamed: 72                           NaN
Unnamed: 73                           NaN
Unnamed: 74                           NaN
Name: 0, Length: 75, dtype: object)


In [119]:
#playing around with way to search and grab values for first sheet 

column_values = current_sheet_raw.columns.values
column_indices = current_sheet_raw.columns

title_string_column_indices = column_indices[1::15]
concentration_value_column_indices = column_indices[2::15]


for index, row in current_sheet_raw.iterrows():
    print((row))
#     dfconc = pd.DataFrame([], columns = [])
    
#     for j in range(len(concentration_value_column_indices)):
        #row prints a SERIES
#         series_1 = row[1]
#         series_2 = row[2]
#         dfnew = pd.concat([series_1, series_2], axis = 1)
        
#         


dfconc

Unnamed: 0                            NaN
from 20200130-3-500KHA-4p78125mgml    NaN
Unnamed: 2                            NaN
Unnamed: 3                            NaN
Unnamed: 4                            NaN
                                     ... 
Unnamed: 70                           NaN
Unnamed: 71                           NaN
Unnamed: 72                           NaN
Unnamed: 73                           NaN
Unnamed: 74                           NaN
Name: 0, Length: 75, dtype: object
Unnamed: 0                                  NaN
from 20200130-3-500KHA-4p78125mgml    conc (um)
Unnamed: 2                               9.5625
Unnamed: 3                              time(s)
Unnamed: 4                                 0.25
                                        ...    
Unnamed: 70                                 NaN
Unnamed: 71                                 NaN
Unnamed: 72                                 NaN
Unnamed: 73                                 NaN
Unnamed: 74            

Name: 114, Length: 75, dtype: object
Unnamed: 0                                         NaN
from 20200130-3-500KHA-4p78125mgml                   5
Unnamed: 2                                        3.01
Unnamed: 3                                      4.1393
Unnamed: 4                                       30.75
                                            ...       
Unnamed: 70                                        NaN
Unnamed: 71                                          5
Unnamed: 72                           3.0701 .. 2.9498
Unnamed: 73                                       5.01
Unnamed: 74                                    1450.35
Name: 115, Length: 75, dtype: object
Unnamed: 0                                         NaN
from 20200130-3-500KHA-4p78125mgml                   6
Unnamed: 2                                        1.69
Unnamed: 3                                     47.8242
Unnamed: 4                                        4.07
                                            ..

Empty DataFrame
Columns: [(nan,), (7,), (0.84,), (14.7875,), (1.81,), (481.7,), (Compound,), (None,), (nan,), (nan,), (nan,), (7,), (0.8491 .. 0.8417,), (0.98,), (383.7027,), (nan,), (nan,), (nan,), (nan,), (nan,), (nan,), (nan,), (nan,), (nan,), (nan,), (nan,), (nan,), (nan,), (nan,), (nan,), (nan,), (nan,), (nan,), (nan,), (nan,), (nan,), (nan,), (nan,), (nan,), (nan,), (nan,), (nan,), (nan,), (nan,), (nan,), (nan,), (7,), (0.84,), (12.3633,), (1.79,), (398.26,), (Compound,), (None,), (nan,), (nan,), (nan,), (7,), (0.8491 .. 0.8417,), (0.94,), (318.6662,), (nan,), (7,), (0.84,), (11.3816,), (1.82,), (371.13,), (Compound,), (None,), (nan,), (nan,), (nan,), (7,), (0.8491 .. 0.8417,), (0.94,), (288.6245,)]
Index: []

[0 rows x 75 columns]

Unnamed: 2     NaN
Unnamed: 17    NaN
Unnamed: 32    NaN
Unnamed: 47    NaN
Unnamed: 62    NaN
Name: 0, dtype: object
Unnamed: 2     9.5625
Unnamed: 17      13.5
Unnamed: 32     11.25
Unnamed: 47     8.128
Unnamed: 62       6.9
Name: 1, dtype: object
Unnamed: 2     ppm
Unnamed: 17    NaN
Unnamed: 32    NaN
Unnamed: 47    ppm
Unnamed: 62    ppm
Name: 2, dtype: object
Unnamed: 2     4.21
Unnamed: 17     NaN
Unnamed: 32     NaN
Unnamed: 47    4.21
Unnamed: 62    4.21
Name: 3, dtype: object
Unnamed: 2     4.13
Unnamed: 17     NaN
Unnamed: 32     NaN
Unnamed: 47    4.13
Unnamed: 62    4.13
Name: 4, dtype: object
Unnamed: 2     3.51
Unnamed: 17     NaN
Unnamed: 32     NaN
Unnamed: 47    3.51
Unnamed: 62    3.51
Name: 5, dtype: object
Unnamed: 2     3.19
Unnamed: 17     NaN
Unnamed: 32     NaN
Unnamed: 47    3.19
Unnamed: 62    3.19
Name: 6, dtype: object
Unnamed: 2     3.01
Unnamed: 17     NaN
Unnamed: 32     NaN
Unnamed: 47    3.01
Unnamed: 62    3.01
Name: 7, dtype: object
Unnamed: 2     1

Name: 107, dtype: object
Unnamed: 2     0.84
Unnamed: 17     NaN
Unnamed: 32     NaN
Unnamed: 47    0.84
Unnamed: 62    0.84
Name: 108, dtype: object
Unnamed: 2     9.5625
Unnamed: 17      13.5
Unnamed: 32     11.25
Unnamed: 47     8.128
Unnamed: 62       6.9
Name: 109, dtype: object
Unnamed: 2     ppm
Unnamed: 17    NaN
Unnamed: 32    NaN
Unnamed: 47    ppm
Unnamed: 62    ppm
Name: 110, dtype: object
Unnamed: 2     4.21
Unnamed: 17     NaN
Unnamed: 32     NaN
Unnamed: 47    4.21
Unnamed: 62    4.21
Name: 111, dtype: object
Unnamed: 2     4.13
Unnamed: 17     NaN
Unnamed: 32     NaN
Unnamed: 47    4.13
Unnamed: 62    4.13
Name: 112, dtype: object
Unnamed: 2     3.51
Unnamed: 17     NaN
Unnamed: 32     NaN
Unnamed: 47    3.51
Unnamed: 62    3.51
Name: 113, dtype: object
Unnamed: 2     3.19
Unnamed: 17     NaN
Unnamed: 32     NaN
Unnamed: 47    3.19
Unnamed: 62    3.19
Name: 114, dtype: object
Unnamed: 2     3.01
Unnamed: 17     NaN
Unnamed: 32     NaN
Unnamed: 47    3.01
Unnamed: 62    

In [32]:
#writing a function to return the column title string that matches a given query
def my_finder_function(df,query):
    string_initializer = []
    for col in df:
        if query in col:
            string_initializer.append(col)
            current_string_initializer = string_initializer[len(string_initializer)-1]
            #fill in the current data for the nth sheet
    return string_initializer

my_finder_function(current_sheet_raw, "from")


['from 20200130-3-500KHA-4p78125mgml',
 'from 20200130-3-500KHA-6p750mgml',
 'from 20200130-3-500KHA-5p625mgml',
 'from 20200130-3-500KHA-4p064mgml',
 'from 20200130-3-500KHA-3p450mgml']

In [22]:
def column_index(df, query_cols):
#     cols = df.columns.values
#     print("current value of cols is:", cols)
#     sidx = np.argsort(cols)
#     print("current value of sidx is:", sidx)
    
    list_initializer = []
    
    #loops through index (column) and rows
    for index, row in df.iterrows():
        #if query word is in the cell:
        if query_cols in index, row:
            print("current cell (row, col) is: ", row, index)
            list_initializer.append(current_sheet_raw.at([row,(index+1)]))

    return list_initializer
#     return sidx[np.searchsorted(cols,query_cols,sorter=sidx)]

column_index(current_sheet_raw, "conc (um)")

# current_sheet_raw.columns.get_loc(current_sheet_raw.columns.contains("from"))
# [current_sheet_raw.columns.get_loc(c) for c in cols if c in current_sheet_raw]

SyntaxError: invalid syntax (<ipython-input-22-b28588aedf5a>, line 12)

In [ ]:
# Use anonymous lambda function and apply method to selectively loop through rows and columns of dataframe looking for different things

#allows us to point to specific cells within each row as inputs to the function, need one function for each desired input
    
# loop through rows, when find 'conc' in one cell grab the value of the cell next to it    
df['X'] = df.apply(lambda row:  , axis=1)

haversine(40.671, -73.985, row['latitude'], row['longitude'])

#need to sweep globally from left to right by column 
#function to append the item to the right of n string to a list initializer
def find_to_the_right(n):
    #find item to the right of n
    list_initializer = []
            if "from" in col:
    
    #assigns number of unique finds to make based on the number of title strings (AKA measurement columns)
    num_unique_finds = len(title_string_initializer)
    
    # Define hour range Boolean arrays
    peak_hours = df.index.hour.isin(range(17, 24))
    shoulder_hours = df.index.hour.isin(range(7, 17))
    off_peak_hours = df.index.hour.isin(range(0, 7))

    # Apply tariffs to hour ranges
    df.iloc[peak_hours, 'cost_cents'] = df.loc[peak_hours, 'energy_kwh'] * 28
    
    df.loc[peak_hours, 'cost_cents'] = df.loc[peak_hours, 'energy_kwh'] * 28
    df.loc[shoulder_hours,'cost_cents'] = df.loc[shoulder_hours, 'energy_kwh'] * 20
    df.loc[off_peak_hours,'cost_cents'] = df.loc[off_peak_hours, 'energy_kwh'] * 12
    
    #search columns from left to right 
    
        #search rows within each column
        
            #if a column and row is identified that contains n
    if n in col:
        colnum =
        rownum =
    
    #append item to the right of n to list initializer
    list_initializer.append(current_sheet_raw.at([rownum,(colnum+1)]))
    
  return lambda a : a * n

concentration_info = find_to_the_right('conc')


mytripler = myfunc(3)

print(mydoubler(11))
print(mytripler(11))

In [503]:
'''The next column label to initialize in the clean dataframe is the 'concentration' for each 'title_string'.
Generalize to one for loop at the end after doing everything well first. Or perhaps all of them we'll see how this goes.'''

#initialize all other column and row initializers for the new dataframe.

concentration_initializer = []
concentration_rows = []
time_initializer = []
time_rows = []
irrad_initializer = []
irrad_rows []
ppm_initializer = []
ppm_rows = []
intensity_initializer = []
intensity_rows = []
range_initializer = []
range_rows = []
absolute_initializer = []
absolute_rows= []

#create dictionary of lists

# test_dict = {'sample_or_control': title_string_df['sample_or_control'], 
#         'replicate_index': title_string_df['replicate_index'], 
#         'title_string': title_string_df['title_string'],
# #         'concentration': [],
# #         'sat_time': [],
# #         'irradiation_bool': [],
# #         'peak_ppm': [],
# #         'peak_intensity': [],
# #         'peak_range': [],
# #         'peak_absolute_integral': [],
#        }

# test_df = pd.DataFrame(test_dict)


#loop through each sheet in the workbook
for n in range(num_sheets):
    
    #read the current book's nth sheet into a Pandas dataframe
    current_sheet_raw = pd.read_excel(b, sheet_name=n)
    
    #assign what sample or control and replicate the book's nth sheet is at
    current_sample_or_control = sample_control_initializer[n]
    current_replicate_index = total_replicate_index[n]
    
    #iterate over all rows and columns per sheet, and assign the right variables to the right lists as you go.
    
    
    
    
    #identify cell that has the string 'conc,' then append the number to the concentration list that is to the RIGHT of the cell, then search down, then search to the right...  search the first row of each sheet and append the 'title_string_initializer' with a title string there is a match
    for col in current_sheet_raw:
        
        if "from" in col:
            title_string_initializer.append(col)
            current_title_string_initializer = title_string_initializer[len(title_string_initializer)-1]
            #fill in the current data for the nth sheet
            title_string_rows.append([current_sample_or_control, current_replicate_index, current_title_string_initializer])
        
title_string_df = pd.DataFrame(title_string_rows, columns=['sample_or_control','replicate_index','title_string'])
            
#determine whether or not to replace the clean book with this dataframe later.... it worked!
title_string_df

SyntaxError: invalid syntax (<ipython-input-503-30f71fc97a9a>, line 11)

In [381]:
current_book_clean

sample_or_control  replicate_index title_string concentration sat_time  \
0            sample                1          NaN           NaN      NaN   
1            sample                2          NaN           NaN      NaN   
2            sample                3          NaN           NaN      NaN   
3           control                1          NaN           NaN      NaN   
4           control                2          NaN           NaN      NaN   
5           control                3          NaN           NaN      NaN   

  irradiation_bool peak_ppm peak_intensity peak_range peak_absolute_integral  
0              NaN      NaN            NaN        NaN                    NaN  
1              NaN      NaN            NaN        NaN                    NaN  
2              NaN      NaN            NaN        NaN                    NaN  
3              NaN      NaN            NaN        NaN                    NaN  
4              NaN      NaN            NaN        NaN                    NaN  
5              NaN      NaN            NaN        NaN                    NaN

In [226]:


#reads the raw data from all sheets in the current book into a single Pandas dataframe object, keeping the sheet indices 

#     pd.concat(pd.read_excel(b, sheet_name = None)
    
#     current_book_raw.to_excel("current_book_raw.xlsx")
    #add important data to the clean dataframe

In [468]:
dict = {'sample_or_control': title_string_df['sample_or_control'], 
        'replicate_index': title_string_df['replicate_index'], 
        'title_string': title_string_df['title_string'],
#         'concentration': [],
#         'sat_time': [],
#         'irradiation_bool': [],
#         'peak_ppm': [],
#         'peak_intensity': [],
#         'peak_range': [],
#         'peak_absolute_integral': [],
       }

# for i, j in test_df.iterrows(): 

test_df = pd.DataFrame(dict)

raw_row_content = []
raw_row_index = []
target_row_content = []
target_row_index = []

for i, j, in current_sheet_raw.iterrows():
    raw_row_content.append(j)
    raw_row_index.append(i)
    
    if "conc" in raw_row_content[i]:
        target_row_content.append(raw_row_content[i])
        target_row_index.append(i)
    
print(raw_row_content)    
raw_df = pd.DataFrame(target_row_content, target_row_index)
raw_df.head(12)

# print(raw_row_index, raw_row_content)
# title_string_rows, columns=['sample_or_control','replicate_index','title_string']

[Unnamed: 0                            NaN
from 20200130-3-500KHA-4p78125mgml    NaN
Unnamed: 2                            NaN
Unnamed: 3                            NaN
Unnamed: 4                            NaN
                                     ... 
Unnamed: 70                           NaN
Unnamed: 71                           NaN
Unnamed: 72                           NaN
Unnamed: 73                           NaN
Unnamed: 74                           NaN
Name: 0, Length: 75, dtype: object, Unnamed: 0                                  NaN
from 20200130-3-500KHA-4p78125mgml    conc (um)
Unnamed: 2                               9.5625
Unnamed: 3                              time(s)
Unnamed: 4                                 0.25
                                        ...    
Unnamed: 70                                 NaN
Unnamed: 71                                 NaN
Unnamed: 72                                 NaN
Unnamed: 73                                 NaN
Unnamed: 74          

Empty DataFrame
Columns: []
Index: []

In [501]:
List = ['conc', 'time', 'irrad']

for index, row in current_sheet_raw.iterrows():
    for word in List:
            target_row_content.append(row[index])
        
        
#         row['Column_name'] = row['Column_name'].partition(word)[0] if (word in row['Column_name']) else row['Column_name']



# for i in current_sheet_raw.itertuples(): 
#     if "conc" in i:
#         target_row_content.append(i)
print(target_row_content)

IndexError: index out of bounds